In [35]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import csv
import tqdm
%matplotlib inline

fdir = '/data/zjc4/'
#Load an image
chip_name = fdir+'train_images/104.tif'
cdir = "/data/zjc4/chipped/train/"
arr = wv.get_image(chip_name)

In [36]:
#Loading our labels
coords1, chips1, classes1 = wv.get_labels(fdir+'xView_train.geojson')

100%|██████████| 601937/601937 [00:04<00:00, 146031.68it/s]


In [38]:
import glob
all_images = glob.glob(fdir+'train_images/*.tif')

In [58]:
selected_labels = [11,12,13,17,18,20,21,19,23,24,25,\
                   28,29,60,61,65,26,41,42,50,40,44,45,47,49]
new_classes = \
{ 
    11:0,
    12:0,
    13:1,
    17:2,
    18:2,
    20:2,
    21:2,
    19:3,
    23:3,
    24:3,
    25:3,
    28:3,
    29:3,
    60:3,
    61:3,
    65:3,
    26:3,
    41:4,
    42:4,
    50:4,
    40:4,
    44:4,
    45:4,
    47:4,
    49:4
}

new_classes = \
{ 
    0:[11,12],
    1:[13],
    2:[17,18,20,21],
    3:[19,23,24,25,28,29,60,61,65,26],
    :[41,42,50,40,44,45,47,49]
}
new_class_strings = ['Small Aircraft','Large Aircraft',\
                     'Small Vehicle','Bus/Truck','Boat']

In [72]:
print("")
#Load the class number -> class string label map
labels = {}
with open('xview_class_labels.txt') as f:
    for row in csv.reader(f):
        labels[int(row[0].split(":")[0])] = row[0].split(":")[1]
        pass
    pass

i = 0
for chip_name in tqdm.tqdm(all_images):
    i+=1
    if i > 500: break
    chip_name = (chip_name.split("/")[-1])
    img_name = chip_name.split(".")[0]

    coords = coords1[chips1==chip_name]
    classes = classes1[chips1==chip_name].astype(np.int64)
    
    c_img, c_box, c_cls = wv.chip_image(img = arr, coords= coords, 
                                        classes=classes, shape=(600,600))
    # xmin,ymin,xmax,ymax = c_box
    #   0    1   2    3
    for c_idx in (range(c_img.shape[0])):
        # Save the chipped image
        c_name = "{:06}_{:02}".format(int(img_name), c_idx)
        # Save the chipped label
        widths = c_box[c_idx][:,2]-c_box[c_idx][:,0]
        x = c_box[c_idx][:,0]+(widths/2)
        
        heights = c_box[c_idx][:,3]-c_box[c_idx][:,1]
        y = c_box[c_idx][:,1]+(heights/2)
        
        szx = c_img[c_idx].shape[0]
        szy = c_img[c_idx].shape[1]
        nwidths,nheights = widths/szx, heights/szy
        nx,ny = x/szx, y/szy

        nheights = np.round(nheights,6)
        nwidths = np.round(nwidths,6)
        nx,ny = np.round(nx,6),np.round(ny,6)
        
        h_cls = c_cls[c_idx]
        data_labels = np.vstack((h_cls,nx,ny,nwidths,nheights)).T
        # Select only valid labels
        y_valid = np.logical_and(nx-nwidths > 0.1,nx+nwidths < 0.9)
        x_valid = np.logical_and(ny-nheights > 0.1,ny+nheights < 0.9)
        cls_valid = (h_cls<30)
        valid = np.logical_and(x_valid,np.logical_and(y_valid,cls_valid))
        data_labels = data_labels[valid,:]
        
        # Break if there are no valid labels(o.w. empty label == error)
        if (np.count_nonzero(valid==True) < 1 ): break
        ff_l = "{}labels/{}.txt".format(cdir,c_name)
        np.savetxt(ff_l, data_labels, fmt='%i %1.6f %1.6f %1.6f %1.6f')
        
        img = Image.fromarray(c_img[c_idx])
        ff_i = "{}images/{}.jpg".format(cdir,c_name)
        img.save(ff_i)





  0%|          | 0/846 [00:00<?, ?it/s]


  0%|          | 1/846 [00:00<02:17,  6.15it/s]




  0%|          | 2/846 [00:00<02:14,  6.28it/s]


  0%|          | 3/846 [00:00<02:20,  5.99it/s]


  0%|          | 4/846 [00:00<02:16,  6.15it/s]


  1%|          | 5/846 [00:00<02:14,  6.27it/s]


  1%|          | 6/846 [00:00<02:14,  6.23it/s]


  1%|          | 7/846 [00:01<02:12,  6.33it/s]


  1%|          | 8/846 [00:01<02:10,  6.40it/s]


  1%|          | 9/846 [00:01<02:25,  5.75it/s]


  1%|          | 10/846 [00:01<02:20,  5.95it/s]


  1%|▏         | 11/846 [00:01<02:19,  5.97it/s]


  1%|▏         | 12/846 [00:01<02:15,  6.14it/s]


  2%|▏         | 13/846 [00:02<02:16,  6.11it/s]


  2%|▏         | 14/846 [00:02<02:13,  6.24it/s]


  2%|▏         | 15/846 [00:02<02:12,  6.29it/s]


  2%|▏         | 16/846 [00:02<02:10,  6.38it/s]


  2%|▏         | 17/846 [00:02<02:09,  6.39it/s]


  2%|▏         | 18/846 [00:02<02:11,  6.29it/s]


  2%|▏         | 19/846 [00:03<02:17,  6.03it/s]


  2%|▏         | 20/846 [00:03<02:13,  6.18it/s]


  2%|▏         | 21/846 [00:03<02:12

KeyboardInterrupt: 

In [77]:
lines = glob.glob(cdir+"images/*")
with open(fdir+"chipped/xview_img.txt", mode='w', encoding='utf-8') as myfile:
    myfile.write('\n'.join(lines))

In [76]:
lines[:10]

['/data/zjc4/chipped/train/images/000091_05.jpg',
 '/data/zjc4/chipped/train/images/001945_02.jpg',
 '/data/zjc4/chipped/train/images/002550_05.jpg',
 '/data/zjc4/chipped/train/images/000091_00.jpg',
 '/data/zjc4/chipped/train/images/001184_00.jpg',
 '/data/zjc4/chipped/train/images/001070_06.jpg',
 '/data/zjc4/chipped/train/images/001070_00.jpg',
 '/data/zjc4/chipped/train/images/000091_14.jpg',
 '/data/zjc4/chipped/train/images/002550_12.jpg',
 '/data/zjc4/chipped/train/images/002440_02.jpg']